# Similarity Search with Databricks

This recipe will show you how to run semantic search with embeddings from Databricks.

## Requirements

1. Weaviate cluster
    1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
    2. [Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded)
    3. [Local deployment](https://weaviate.io/developers/weaviate/installation/docker-compose#starter-docker-compose-file)
    4. [Other options](https://weaviate.io/developers/weaviate/installation)

2. Databricks token

3. Endpoint to an embedding model on Databricks

## Import Dependencies, Libraries, and Keys

In [ ]:
!pip install --q weaviate-client

In [1]:
import weaviate, os
from weaviate.embedded import EmbeddedOptions
import weaviate.classes as wvc
import weaviate.classes.config as wc
import requests, json
import weaviate.classes.query as wq

## Connect to Weaviate

Only choose one option from the below.

**Weaviate Cloud Deployment**

In [2]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
DATABRICKS_TOKEN = os.environ["DATABRICKS_TOKEN"] # Replace with your Databricks token

# Weaviate Cloud Deployment
client = weaviate.connect_to_wcs(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={"X-Databricks-Token": DATABRICKS_TOKEN}
)

print(client.is_ready())

True


**Embedded Weaviate**

In [ ]:
# DATABRICKS_TOKEN = os.environ["DATABRICKS_TOKEN"] # Replace with your Databricks token

# client = weaviate.WeaviateClient(
#     embedded_options=EmbeddedOptions(
#         version="1.30.0",
#         additional_env_vars={
#             "ENABLE_MODULES": "text2vec-databricks"
#         }),
#         additional_headers={
#             "X-Databricks-Token": DATABRICKS_TOKEN
#         }
# )

# client.connect()

**Local Deployment**

In [ ]:
# DATABRICKS_TOKEN = os.environ["DATABRICKS_TOKEN"] # Replace with your Databricks token

# client = weaviate.connect_to_local(
#   headers={
#     "X-Databricks-Token": DATABRICKS_TOKEN
#   }
# )
# print(client.is_ready())

## Create a collection
> Collection stores your data and vector embeddings.

In [6]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.

EMBEDDING_ENDPOINT = os.environ["EMBEDDING_ENDPOINT"] # Replace with the endpoint to your embedding model

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_databricks(
        endpoint=EMBEDDING_ENDPOINT, # endpoint to the embedding model
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


{"build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version":"1.30.0","level":"warning","msg":"prop len tracker file /Users/erikacardenas/.local/share/weaviate/jeopardyquestion/ltfNgZSUE4cS/proplengths does not exist, creating new tracker","time":"2025-05-02T15:20:08-04:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version":"1.30.0","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2025-05-02T15:20:08-04:00","wait_for_cache_prefill":false}
{"build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version":"1.30.0","level":"info","msg":"Created shard jeopardyquestion_ltfNgZSUE4cS in 4.476375ms","time":"2025-05-02T15:20:08-04:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version"

## Import the Data

In [7]:
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

Insert complete.


## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### `nearText` Example
Find a `JeopardyQuestion` about "animals in movies". Limit it to only 4 responses.

In [8]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="african beasts",
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: 77733229-d9b6-4424-8265-58f4e3d61feb
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 

ID: 5bdaf211-cce3-40ae-96ba-04cfb6362c8a
Data: {
  "answer": "Elephant",
  "question": "It's the only living mammal in the order Proboseidea",
  "category": "ANIMALS"
} 

ID: 074d10c6-9ffe-4018-9314-735bdebac37c
Data: {
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
} 

ID: 1aca8085-3eb8-4b7e-8e05-07987fa5fe3a
Data: {
  "answer": "species",
  "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  "category": "SCIENCE"
} 



**Return vector embeddings**

In [9]:
response = jeopardy.query.near_text(
    query="african beasts",
    include_vector=True,
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector, "\n")

ID: 77733229-d9b6-4424-8265-58f4e3d61feb
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
}
Vector: {'default': [0.0258941650390625, 0.2464599609375, -0.10943603515625, 0.267578125, 0.0718994140625, -0.059356689453125, -0.380859375, 0.3798828125, 0.7783203125, -0.2174072265625, -0.142578125, -0.80029296875, 0.2445068359375, 0.91162109375, -0.288818359375, 0.119140625, -0.453125, 0.11419677734375, 0.245849609375, 0.5498046875, -0.1651611328125, -0.6767578125, 0.337646484375, 0.316162109375, 0.75732421875, -0.80859375, 0.03851318359375, -0.57373046875, 0.5986328125, 0.419921875, -0.07861328125, -0.413818359375, 0.17041015625, 0.1231689453125, -0.16650390625, -0.333984375, 0.10357666015625, 1.65234375, 0.064453125, 0.01142120361328125, -0.316650390625, 0.2425537109375, -1.0263671875, 0.98876953125, -0.339599609375, -0.07330322265625, 0.10467529296875, 0.52490234375, -0.66796875, 1.0234

**Now, also request the `distance` for each returned item.**

In [10]:
response = jeopardy.query.near_text(
    query="african beasts",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Distance:", item.metadata.distance)
    print("Data:", item.properties, "\n")

ID: 77733229-d9b6-4424-8265-58f4e3d61feb
Distance: 0.25670284032821655
Data: {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'} 

ID: 5bdaf211-cce3-40ae-96ba-04cfb6362c8a
Distance: 0.41141563653945923
Data: {'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'} 

ID: 074d10c6-9ffe-4018-9314-735bdebac37c
Distance: 0.4607570171356201
Data: {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'} 

ID: 1aca8085-3eb8-4b7e-8e05-07987fa5fe3a
Distance: 0.5187289118766785
Data: {'answer': 'species', 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'} 



### `nearObject` Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id `a1dd67f9-bfa7-45e1-b45e-26eb8c52e9a6`. (The id was taken from the query above)

In [12]:
response = jeopardy.query.near_object(
    near_object="77733229-d9b6-4424-8265-58f4e3d61feb", # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

ID: 77733229-d9b6-4424-8265-58f4e3d61feb
Data: {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'} 

ID: 5bdaf211-cce3-40ae-96ba-04cfb6362c8a
Data: {'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'} 

ID: 0b2c85c9-e554-4720-a390-f0cf98f552cf
Data: {'answer': 'the diamondback rattler', 'question': 'Heaviest of all poisonous snakes is this North American rattlesnake', 'category': 'ANIMALS'} 

ID: 074d10c6-9ffe-4018-9314-735bdebac37c
Data: {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'} 



### `nearVector` Example
Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [ ]:
response = jeopardy.query.near_vector(
    near_vector=[-0.0125526935, -0.021168863, ... ], # replace with a vector object in your cluster
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")